In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision


import numpy as np
import matplotlib.pyplot as plt

from data.EuroNotes import EuroNotes
from models.ResNet18 import pretrained_res18

In [2]:
# # # # # # # # # # # # # # # #
# Load Classifier Models
# # # # # # # # # # # # # # # #


print('Loading base classifier model and robust model')
model_base = pretrained_res18()
model_robust = pretrained_res18()
print('Loaded')

Loading base classifier model and robust model
Loaded


In [3]:
means = np.array([ 0.14588552,  0.26887908,  0.14538361])
stds = np.array([ 0.20122388,  0.2800698 ,  0.20029236])
transformations = transforms.Compose([transforms.ToTensor(), transforms.Normalize(means, stds)])

train_set = EuroNotes('../data-augmentation/banknotes_augmented/train', transform=transformations, resize=False)
val_set = EuroNotes('../data-augmentation/banknotes_augmented/val', transform=transformations, resize=False)

train_loader = DataLoader(train_set, batch_size=25, shuffle=True, num_workers=16)
val_loader = DataLoader(val_set, batch_size=25, shuffle=True, num_workers=16)

In [4]:
import matplotlib.gridspec as gridspec

gs = gridspec.GridSpec(2, 2)
plt.figure(figsize=(20,10))

for i_batch, data in enumerate(train_loader):
    if i_batch > 3:
        break
    images = Variable(data['image'])
    print(images.shape)
    print(type(images))
    preds = model_base(images)
    print(preds[:3, :])
#     plt.title(i_batch)
#     im = images[1, :, :, :].data.numpy() * stds.reshape((3, 1, 1))
    
#     im = im + means.reshape((3, 1, 1))
#     im = im.astype(float)
#     im = im.astype(float)
#     im -= np.min(im)
#     im /= np.max(im)
#     im = np.swapaxes(np.swapaxes(im, 0, 2).reshape((256, 256, 3)), 0, 1)
#     ax = plt.subplot(gs[int(i_batch/2), i_batch % 2])
#     plt.imshow(im)
# plt.show()
    

torch.Size([25, 3, 256, 256])
<class 'torch.autograd.variable.Variable'>
Variable containing:
 -8.8892 -12.9246 -12.4907  12.0369
-15.2968  21.3529 -16.6540 -25.3030
-10.7494 -10.8634 -10.9365  14.2068
[torch.FloatTensor of size 3x4]

torch.Size([25, 3, 256, 256])
<class 'torch.autograd.variable.Variable'>
Variable containing:
-12.5420  14.8742 -14.8526 -12.5986
-15.7929 -14.9017  22.5197 -16.9204
 14.5632 -12.3667 -11.7706 -14.2410
[torch.FloatTensor of size 3x4]

torch.Size([25, 3, 256, 256])
<class 'torch.autograd.variable.Variable'>
Variable containing:
-13.0031  11.1113 -10.3556 -12.0493
 11.6338  -9.5876  -8.5825 -14.3612
 -8.1341  -6.4036 -10.4267   7.5136
[torch.FloatTensor of size 3x4]

torch.Size([25, 3, 256, 256])
<class 'torch.autograd.variable.Variable'>
Variable containing:
 -7.1234 -10.9897  10.5409 -12.3497
-13.9428 -14.5478 -16.0937  18.7252
-11.7195 -20.2678  26.6282 -22.1263
[torch.FloatTensor of size 3x4]



In [5]:
from attackers.BlackBoxAttacker import GANAttack

In [6]:
ganatt = GANAttack((3, 256, 256), 0.01)

Loading pre-trained GANAttack


In [7]:
images.shape

torch.Size([25, 3, 256, 256])

In [8]:
preds = model_base(images)
print(preds.shape)

torch.Size([25, 4])


In [9]:
np.random.uniform(-1, 0.1, (3, 2))

array([[-0.23771642, -0.09131742],
       [-0.29995057, -0.20065749],
       [-0.25190148, -0.07366671]])